# Model for Nature Conservancy Fisheries Kaggle Competition

#### Dependencies

In [ ]:
import fish_data as fd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd
import json

#### Helper functions

In [ ]:
help(fd)

#### Generate a list of filenames

In [ ]:
fish_filenames = fd.generate_filenames_list('data/train/', subfolders = True)
print("There are {} filenames in the master set list".format(len(fish_filenames)))
test_filenames = fd.generate_filenames_list('data/test_stg1/', subfolders = False)
print("There are {} filenames in the test set list".format(len(test_filenames)))

#### Retrieve Dictionary of image dimensions

In [ ]:
with open('dimensions_dict.json') as f:
    dim_dict = json.load(f)
    
print("Training/Valid set filename dimensions downloaded correctly: {}".format(
        dim_dict.get(fish_filenames[0]) == [720, 1280, 3]))
print("Training/Valid set filename dimensions downloaded correctly: {}".format(
        dim_dict.get(test_filenames[0]) == [720, 1280, 3]))

#### Generate the labels for the master set list

In [ ]:
fish_label_arr = fd.make_labels(fish_filenames, 'train/', '/img')
fish_label_arr.shape
print("One label per row entry: {}".format(all(np.sum(fish_label_arr, 1) == 1) ))

#### Shuffle and split the master set list into training and validation sets

In [ ]:
valid_size = 300
files_train, files_val, y_train, y_val = train_test_split(fish_filenames, fish_label_arr, test_size = valid_size)
print("Validation set size: {}".format(y_val.shape[0]))
print("Training set size: {}".format(y_train.shape[0]))

#### Generate a files_train list that represents each class of fish equally

In [ ]:
"""Need to refactor generate_balanced_filenames to work from this list, not from scratch."""

In [ ]:
train_dims_list = []
for f in files_train :
    train_dims_list.append(dim_dict.get(f))

## Graph and Session Runs

#### Graph parameters

In [ ]:
%run -i 'PARAMETERS.py'

#### Session parameters

In [ ]:
version_ID = 'v2.0.0.2'

In [ ]:
%run -i 'GRAPH.py'

In [ ]:
%run -i 'SESSION.py'

#### Notes during run 


In [ ]:
print(test_df)

In [ ]:
W.shape

In [ ]:
for i in range(W.shape[3]) :
    print(i)
    plt.imshow(W[:,:,:,i])
    plt.show()